In [7]:
from __future__ import print_function

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop, Adadelta, Adagrad, Adam, Nadam, SGD
from keras.callbacks import EarlyStopping, TerminateOnNaN
from keras import backend as K
from keras.losses import mean_squared_error
import tensorflow as tf

# Normalize the data.
from sklearn import preprocessing
from keras.regularizers import l1_l2

import random

def pass_arg(nsim, tr_size, dropoutrate):
    print("Tr_size:", tr_size)
    def fix_seeds(seed):
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
        sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
        # K.set_session(sess)
        tf.compat.v1.keras.backend.set_session(sess)

    ss = 1
    fix_seeds(ss)

    # MC dropout
    class MCDropout(Dropout):
        def call(self, inputs, training=None):
            return super(MCDropout, self).call(inputs, training=True)

    # Compute the RMSE given the ground truth (y_true) and the predictions(y_pred)
    def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1))

    # Making sure final porosity is less than initial
    def poros(poroi, porof):
        porofn = -porof*(porof<0)
        porofp = porof*(porof>=poroi) - poroi*(porof>=poroi)
        return porofp+porofn

    def phy_loss_mean(params):
        # useful for cross-checking training
        loss1, loss2, loss3, loss4, lam1, lam2 = params
        x1, x2, x3 = loss1*(loss1>0), loss2*(loss2>0), loss3*(loss3>0)
    #     print(np.mean(x1), x1.shape[0])
    #     print(np.mean(x2), x2.shape[0])
    #     print(np.mean(x3), x3.shape[0])

        if x1.any() and x1.shape[0]>1:
            X_scaled1 = (x1 - np.min(x1)) / (np.max(x1) - np.min(x1))
            x1 = X_scaled1
        if x2.any() and x2.shape[0]>1:
            X_scaled2 = (x2 - np.min(x2)) / (np.max(x2) - np.min(x2))
            x2 = X_scaled2
        if x3.any() and x3.shape[0]>1:
            X_scaled3 = (x3 - np.min(x3)) / (np.max(x3) - np.min(x3))
            x3 = X_scaled3
        return (lam1*np.mean(x1) + lam2*np.mean(x2) + lam2*np.mean(x3))
    #     return (lam1*np.mean(x1) + lam2*np.mean(x2) + lam2*np.mean(x3) + lam2*loss4)

    def PGNN_train_test(optimizer_name, optimizer_val, drop_rate, iteration, n_layers, n_nodes, tr_size, lamda, reg):

        # Hyper-parameters of the training process
        # batch_size = int(tr_size/2)
        batch_size = 10
        num_epochs = 300
        val_frac = 0.25
        patience_val = 80

        # Initializing results filename
        exp_name = optimizer_name + '_drop' + str(drop_rate) + '_nL' + str(n_layers) + '_nN' + str(n_nodes) + '_trsize' + str(tr_size) + '_iter' + str(iteration)
        exp_name = exp_name.replace('.','pt')
        results_dir = '../results/'
        model_name = results_dir + exp_name + '_NoPhyInfomodel.h5' # storing the trained model
        if reg:
            results_name = results_dir + exp_name + '_results_regularizer.dat' # storing the results of the model
        else:
            results_name = results_dir + exp_name + '_results.dat' # storing the results of the model

        # Load labeled data
        data = np.loadtxt('../data/labeled_data.dat')
    #     data = np.loadtxt('../data/labeled_data_BK_constw_unique.dat')
    #     data = np.loadtxt('../data/labeled_data_BK_constw_v2.dat')
        # x_labeled = data[:, :-5] # -2 because we do not need porosity predictions
        x_labeled = data[:, :2] # -2 because we do not need porosity predictions
        y_labeled = data[:, -2:-1]

        # normalize dataset with MinMaxScaler
        scaler = preprocessing.MinMaxScaler(feature_range=(0, 1.0))
    #     scaler = preprocessing.StandardScaler()
        x_labeled = scaler.fit_transform(x_labeled)
        # y_labeled = scaler.fit_transform(y_labeled)

        # train and test data
        trainX, trainY = x_labeled[:tr_size,:], y_labeled[:tr_size]
    #     testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]
    #     init_poro = data[tr_size:, -1]
        testX, testY = x_labeled[tr_size:,:], y_labeled[tr_size:]
        init_poro = data[tr_size:, -1]

        # Creating the model
        model = Sequential()
        for layer in np.arange(n_layers):
            if layer == 0:
                model.add(Dense(n_nodes, activation='relu', input_shape=(np.shape(trainX)[1],)))
            else:
                if reg:
                    model.add(Dense(n_nodes, activation='relu', kernel_regularizer=l1_l2(l1=.001, l2=.001)))
                else:
                    model.add(Dense(n_nodes, activation='relu'))
            model.add(MCDropout(rate=drop_rate))
        model.add(Dense(1, activation='linear'))

        model.compile(loss='mean_squared_error',
                      optimizer=optimizer_val,
                      metrics=[root_mean_squared_error])

        early_stopping = EarlyStopping(monitor='val_loss', patience=patience_val,verbose=1)

        print('Running...' + optimizer_name)
        history = model.fit(trainX, trainY,
                            batch_size=batch_size,
                            epochs=num_epochs,
                            verbose=0,
                            validation_split=val_frac, callbacks=[early_stopping, TerminateOnNaN()])

        test_score = model.evaluate(testX, testY, verbose=1)
        print(test_score)

#         samples = []
        test_scores = []
        for i in range(int(nsim)):
#             print("simulation num:",i)
#             predictions = model.predict(testX)
#             samples.append(predictions)
            test_score = model.evaluate(testX, testY, verbose=1)
            test_scores.append(test_score[1])
        return np.array(test_scores)



    # Main Function
    if __name__ == '__main__':

        fix_seeds(1)

        # List of optimizers to choose from    
        optimizer_names = ['Adagrad', 'Adadelta', 'Adam', 'Nadam', 'RMSprop', 'SGD', 'NSGD']
        optimizer_vals = [Adagrad(clipnorm=1), Adadelta(clipnorm=1), Adam(clipnorm=1), Nadam(clipnorm=1), RMSprop(clipnorm=1), SGD(clipnorm=1.), SGD(clipnorm=1, nesterov=True)]

        # selecting the optimizer
        optimizer_num = 1
        optimizer_name = optimizer_names[optimizer_num]
        optimizer_val = optimizer_vals[optimizer_num]

        # Selecting Other Hyper-parameters
        drop_rate = dropoutrate # Fraction of nodes to be dropped out
        n_layers = 2 # Number of hidden layers
        n_nodes = 5 # Number of nodes per hidden layer

        # # Iterating over different training fractions and splitting indices for train-test splits
        # trsize_range = [4,6,8,10,20]

        # #default training size = 5000
        # tr_size = trsize_range[4]

        tr_size = int(tr_size)

        # use regularizer
        reg = True

        #set lamda=0 for pgnn0
        lamda = [1, 1] # Physics-based regularization constant

        # total number of runs
        iter_range = np.arange(1)
        testrmse=[]
        # iterating through all possible params
        for iteration in iter_range:
            # results, result_file, pred, obs, rmse = PGNN_train_test(optimizer_name, optimizer_val, drop_rate, 
                            # iteration, n_layers, n_nodes, tr_size, lamda, reg)
            # testrmse.append(rmse)
            pred = PGNN_train_test(optimizer_name, optimizer_val, drop_rate, 
                            iteration, n_layers, n_nodes, tr_size, lamda, reg)
    
    return np.squeeze(pred)


In [8]:
mean_rmses=[]
std_rmses=[]
for ii in ([.005,.01,.02,.05,.1,.15,.2,.25,.3,0.5]):
    test_rmse = pass_arg(50, 30, ii)
    mean_rmse = np.mean(test_rmse)
    std_rmse = np.std(test_rmse)
    mean_rmses.append(mean_rmse)
    std_rmses.append(std_rmse)
    
mean_rmses

Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 0us/step
[0.0007204551948234439, 0.007647312246263027]
9/9 [==============================] - 0s 0us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 0us/step
[0.0007808288210071623, 0.007887277752161026]
9/9 [==============================] - 0s 110us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 111us/step
[0.000673063041176647, 0.00897886324673891]
9/9 [==============================] - 0s 0us/step


9/9 [==============================] - 0s 111us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 0us/step
[0.0007548031862825155, 0.013644075021147728]
9/9 [==============================] - 0s 223us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 111us/step
[0.0007085137185640633, 0.013792681507766247]
9/9 [==============================] - 0s 111us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 0us/step
[0.0006968736415728927, 0.015226433984935284]
9/9 [==============================] - 0s 111us/step


9/9 [==============================] - 0s 111us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 111us/step
[0.0007127012358978391, 0.016155634075403214]
9/9 [==============================] - 0s 111us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 0us/step
[0.0006831486825831234, 0.015445700846612453]
9/9 [==============================] - 0s 0us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 111us/step
[0.000711671425960958, 0.01522129587829113]
9/9 [==============================] - 0s 111us/step


9/9 [==============================] - 0s 111us/step
Tr_size: 30
Running...Adadelta
9/9 [==============================] - 0s 111us/step
[0.0012736035278066993, 0.016610365360975266]
9/9 [==============================] - 0s 111us/step


[0.008075667396187782,
 0.008730287104845047,
 0.00945755759254098,
 0.010691170282661915,
 0.013108328711241483,
 0.015919972322881223,
 0.016306096762418745,
 0.016108683384954928,
 0.016140754595398904,
 0.016364065427333117]

In [9]:
mean_rmses

[0.008075667396187782,
 0.008730287104845047,
 0.00945755759254098,
 0.010691170282661915,
 0.013108328711241483,
 0.015919972322881223,
 0.016306096762418745,
 0.016108683384954928,
 0.016140754595398904,
 0.016364065427333117]

In [10]:
std_rmses

[0.0019613543848239364,
 0.0022927945145296675,
 0.002307015927800145,
 0.002528065212618427,
 0.002707605131242183,
 0.0017640548255215694,
 0.0015982525555376058,
 0.0012499834945529024,
 0.0012423847723920653,
 0.0011461824117685085]

In [6]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mean_rmses, "../mean_rmse_dnn_MC.dat")
save_obj(std_rmses, "../std_rmse_dnn_MC.dat")


In [4]:
mc_pred=np.mean(pred,axis=0)
np.mean(mc_pred)

0.02413137

In [5]:
np.mean(np.std(pred,axis=1))

0.015234125

In [544]:
# list1 = [0.01,0.02,0.05,0.1, 0.2, 0.25,0.3,0.35]
# list2 = [5,6,7,8,9,10,11,12,15,20]

# from collections import OrderedDict
# param_grid = OrderedDict(rate = list1, node = list2)
    
# import itertools as it
# allNames = sorted(param_grid)
# combinations = it.product(*(param_grid[Name] for Name in allNames))
# param_combo_list = list(combinations)
# print(len(param_combo_list))
    
# std_list = []
# for it in np.arange(len(param_combo_list)):
#     pred = pass_arg(50, 20, param_combo_list[it])
#     mc_pred=np.mean(pred,axis=0)
#     std_list.append(np.std(pred,axis=1))
# #     print(param_combo_list[it],np.mean(mc_pred_sd))

In [545]:
# stdl = np.array(std_list)
# np.sort(np.mean(stdl,axis=1))

In [546]:
# stdl = np.array(std_list)
# # np.mean(stdl,axis=1)
# argss=np.argsort(np.mean(stdl,axis=1))
# combo = np.array(param_combo_list)
# combo[list(argss)]
# # pred

In [547]:
# mc_pred=np.mean(pred,axis=0)
# mc_pred_sd=np.std(pred,axis=1)
# np.mean(mc_pred_sd)

In [548]:
import pickle

def save_obj(obj, name):
    with open(name, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
        
save_obj(mc_pred, "../pred_dnn_MC_Xx1.dat")